In [7]:
require 'torch'
require 'optim'

print '==> Processing options'

cmd = torch.CmdLine()
cmd:text()
cmd:text('Reducing False Alarms using CNNs')
cmd:text()
cmd:text('Options:')
-- Data: 'chal', 'mimic+chal_all', 'mimic+chal_small'
cmd:option('-datatype', 'chal')
-- Model
cmd:option('-nTarget', 2)
cmd:option('-nInputFeature', 1)
cmd:option('-inputSize', 2500) -- 250Hz * 10sec
--- For convolutional networks
cmd:option('-convlayer_num', 3)
cmd:option('-nFeatures_c', 75)
--- For MLP
cmd:option('-mlplayer_num', 2)
cmd:option('-nUnit_mlp', 500)
-- Experiment Setting
cmd:option('-dbseed', 1)
cmd:option('-weightseed', 1)
cmd:option('-batchsize', 10)
cmd:option('-nFold', 5)
cmd:option('-maxIter', 300)
cmd:option('-lr_sup', 0.001, 'Learning rate')
cmd:option('-lrdecay',1e-5, 'Learning rate decay')
cmd:option('-momentum', 0)
cmd:option('-dropout_rate', 0.5)
cmd:option('-pretraining', false)
-- Conv Setting
cmd:option('-kernel', 25)
cmd:option('-pool', 5)
-- Torch Setting
cmd:option('-thread', 16)
-- File name
cmd:option('-foldername', '/home/heehwan/Workspace/Data/ReduceFA_2015/cnn_output/weirdmimic/')
cmd:option('-filename', 'experiment_01/' .. 'itorch_test')

cmd:text()
option = cmd:parse(arg or {})

torch.setnumthreads(option.thread)

==> Processing options	


In [8]:
require 'getLearningData'
nTraining, trainset_input, trainset_target, nTesting, testset_input, testset_target = getLearningData(option)

==> Load datasets	


In [9]:
require 'buildCNNModel'
model = buildCNNModel(option)

==> Construct CNN model	


nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> output]
  (1): nn.SpatialConvolutionMM(1 -> 75, 1x25)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(1,5,1,5)
  (4): nn.SpatialConvolutionMM(75 -> 75, 1x25)
  (5): nn.ReLU
  (6): nn.SpatialMaxPooling(1,5,1,5)
  (7): nn.SpatialConvolutionMM(75 -> 75, 1x25)
  (8): nn.ReLU
  (9): nn.SpatialMaxPooling(1,5,1,5)
  (10): nn.Reshape(1050)
  (11): nn.Linear(1050 -> 500)
  (12): nn.ReLU
  (13): nn.Dropout(0.500000)
  (14): nn.Linear(500 -> 500)
  (15): nn.ReLU
  (16): nn.Dropout(0.500000)
  (17): nn.Linear(500 -> 2)
  (18): nn.LogSoftMax
}
{
  gradInput : CudaTensor - empty
  modules : 
    {
      1 : 
        nn.SpatialConvolutionMM(1 -> 75, 1x25)
        {
          dH : 1
          dW : 1
          nOutputPlane : 75
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
          fgradInput : CudaTensor - empty


fgradInput : CudaTensor - empty
          finput : CudaTensor - empty
          gradBias : CudaTensor - size: 75
          padH : 0
          weight : CudaTensor - size: 75x1875
          bias : CudaTensor - size: 75
          gradWeight : CudaTensor - size: 75x1875
          padW : 0
          nInputPlane : 75
          kW : 1
          kH : 25
        }
      5 : 
        nn.ReLU
        {
          inplace : false
          threshold : 0
          val : 0
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
        }
      6 : 
        nn.SpatialMaxPooling(1,5,1,5)
        {
          dH : 5
          dW : 1
          padH : 0
          gradInput : CudaTensor - empty
          indices : CudaTensor - empty
          kH : 5
          ceil_mode : false
          output : CudaTensor - empty
          padW : 0
          kW : 1
        }
      7 : 
        nn.SpatialConvolutionMM(75 -> 75, 1x25)
        {
          dH : 1
          dW : 1
          nOutputPlane :

         threshold : 0
          val : 0
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
        }
      13 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : true
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - empty
        }
      14 : 
        nn.Linear(500 -> 500)
        {
          gradBias : CudaTensor - size: 500
          weight : CudaTensor - size: 500x500
          bias : CudaTensor - size: 500
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 500x500
          output : CudaTensor - empty
        }
      15 : 
        nn.ReLU
        {
          inplace : false
          threshold : 0
          val : 0
          output : CudaTensor - empty
          gradInput : CudaTensor - empty
        }
      16 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
        

In [10]:
print '==> Defining loss'

criterion = nn.ClassNLLCriterion()
criterion:cuda()
----------------------------------------------------------------------
print '==> Defining some tools'

classes = {'False Alarm','True Alarm'}
confusion = optim.ConfusionMatrix(classes)
----------------------------------------------------------------------
print '==> configuring optimizer'

optimState = {
learningRate = option.lr_sup,
weightDecay = 0,
momentum = option.momentum,
learningRateDecay = option.lrdecay
}
optimMethod = optim.sgd
----------------------------------------------------------------------
print '==> Defining training procedure'

parameters, gradParameters = model:getParameters()
batchsize = option.batchsize

train_accu = torch.zeros(option.maxIter, 1)
train_err = torch.zeros(option.maxIter, 1)

test_accu = torch.zeros(option.maxIter, 1)
test_err = torch.zeros(option.maxIter, 1)
test_conf = torch.zeros(option.maxIter, 4)

Maxiter = option.maxIter

==> Defining loss	


==> Defining some tools	
==> configuring optimizer	
==> Defining training procedure	


In [14]:
m = model.modules[1]

In [15]:
itorch.image(m.weight)

In [16]:
print '==> Start training'

require 'training_cnn'
require 'testing_cnn'
iter = 1
while iter <= Maxiter do
    train()
    test()
    m = model.modules[1]
    itorch.image(m.weight)
    iter = iter + 1
end

==> Start training	

==> doing epoch on training data:	
==> online epoch # 1 [batchSize = 10]	


==> time to learn 1 sample = 1.4970699946086ms	
ConfusionMatrix:
[[      61     307]   16.576% 	[class: False Alarm]
 [      53     179]]  77.155% 	[class: True Alarm]
 + average row correct: 46.865630149841% 
 + average rowUcol correct (VOC measure): 23.849479854107% 
 + global correct: 40%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.23849479854107
  _targ_idx : LongTensor - empty
  averageValid : 0.46865630149841
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 0.4
}
==> current error = 0.69810443202655	

==> testing on test set:	


==> time to test 1 sample = 0.60253217816353ms	
ConfusionMatrix:
[[       6      72]   7.692% 	[class: False Alarm]
 [       3      47]]  94.000% 	[class: True Alarm]
 + average row correct: 50.846153870225% 
 + average rowUcol correct (VOC measure): 22.965998202562% 
 + global correct: 41.40625%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.22965998202562
  _targ_idx : LongTensor - empty
  averageValid : 0.50846153870225
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
 

 _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 0.4140625
}



==> doing epoch on training data:	
==> online epoch # 2 [batchSize = 10]	


...

==> time to learn 1 sample = 1.3731082280477ms	
ConfusionMatrix:
[[     368       0]   100.000% 	[class: False Alarm]
 [       1     231]]  99.569% 	[class: True Alarm]
 + average row correct: 99.784481525421% 
 + average rowUcol correct (VOC measure): 99.648979306221% 
 + global correct: 99.833333333333%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.99648979306221
  _targ_idx : LongTensor - empty
  averageValid : 0.99784481525421
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 0.99833333333333
}
==> current error = 0.013288265764713	

==> testing on test set:	


==> time to test 1 sample = 0.56172721087933ms	
ConfusionMatrix:
[[      73       5]   93.590% 	[class: False Alarm]
 [      13      37]]  74.000% 	[class: True Alarm]
 + average row correct: 83.794870972633% 
 + average rowUcol correct (VOC measure): 73.746255040169% 
 + global correct: 85.9375%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.73746255040169
  _targ_idx : LongTensor - empty
  averageValid : 0.83794870972633
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 0.859375
}



==> doing epoch on training data:	
==> online epoch # 300 [batchSize = 10]	


==> time to learn 1 sample = 1.3890600204468ms	
ConfusionMatrix:
[[     368       0]   100.000% 	[class: False Alarm]
 [       0     232]]  100.000% 	[class: True Alarm]
 + average row correct: 100% 
 + average rowUcol correct (VOC measure): 100% 
 + global correct: 100%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 1
  _targ_idx : LongTensor - empty
  averageValid : 1
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 1
}
==> current error = 0.010822866161664	

==> testing on test set:	


==> time to test 1 sample = 0.5678366869688ms	
ConfusionMatrix:
[[      70       8]   89.744% 	[class: False Alarm]
 [      11      39]]  78.000% 	[class: True Alarm]
 + average row correct: 83.871793746948% 
 + average rowUcol correct (VOC measure): 72.946530580521% 
 + global correct: 85.15625%
{
  valids : FloatTensor - size: 2
  mat : LongTensor - size: 2x2
  averageUnionValid : 0.72946530580521
  _targ_idx : LongTensor - empty
  averageValid : 0.83871793746948
  classes : 
    {
      1 : False Alarm
      2 : True Alarm
    }
  _prediction : FloatTensor - size: 2
  _pred_idx : LongTensor - size: 1
  nclasses : 2
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 2
  totalValid : 0.8515625
}


In [17]:
print(m.weight:size())


 75
 25
[torch.LongStorage of size 2]

